In [12]:
#%pip install kagglehub
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

In [13]:
# Example
file_path = "written_name_test_v2.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "landlord/handwriting-recognition",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

C:\Users\adam4\AppData\Local\Temp\ipykernel_31316\2912609907.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


First 5 records:         FILENAME  IDENTITY
0  TEST_0001.jpg     KEVIN
1  TEST_0002.jpg  CLOTAIRE
2  TEST_0003.jpg      LENA
3  TEST_0004.jpg     JULES
4  TEST_0005.jpg   CHERPIN


In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("landlord/handwriting-recognition")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\adam4\.cache\kagglehub\datasets\landlord\handwriting-recognition\versions\1


In [15]:
train = pd.read_csv('1/written_name_train_v2.csv')
valid = pd.read_csv('1/written_name_validation_v2.csv')
train

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE
...,...,...
330956,TRAIN_330957.jpg,LENNY
330957,TRAIN_330958.jpg,TIFFANY
330958,TRAIN_330959.jpg,COUTINHO DESA
330959,TRAIN_330960.jpg,MOURAD


In [16]:
print("Number of NaNs in train set      : ", train['IDENTITY'].isnull().sum())
print("Number of NaNs in validation set : ", valid['IDENTITY'].isnull().sum())
train.dropna(axis=0, inplace=True)
valid.dropna(axis=0, inplace=True)
print("Number of NaNs in train set      : ", train['IDENTITY'].isnull().sum())
print("Number of NaNs in validation set : ", valid['IDENTITY'].isnull().sum())

Number of NaNs in train set      :  565
Number of NaNs in validation set :  78
Number of NaNs in train set      :  0
Number of NaNs in validation set :  0


In [21]:
#bad = train[train['IDENTITY' == "UNREADABLE"]]
#bad.reset_index(inplace = True, drop =True)

In [22]:
train = train[train["IDENTITY"] != 'UNREADABLE']
valid = valid[valid["IDENTITY"] != "UNREADABLE"]
train['IDENTITY'] = train['IDENTITY'].str.upper()
valid['IDENTITY'] = valid['IDENTITY'].str.upper()
train.reset_index(inplace=True, drop=True)
valid.reset_index(inplace=True, drop=True)

C:\Users\adam4\AppData\Local\Temp\ipykernel_31316\2334786774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['IDENTITY'] = train['IDENTITY'].str.upper()


In [23]:
train

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE
...,...,...
330289,TRAIN_330957.jpg,LENNY
330290,TRAIN_330958.jpg,TIFFANY
330291,TRAIN_330959.jpg,COUTINHO DESA
330292,TRAIN_330960.jpg,MOURAD


In [24]:
import os
from difflib import SequenceMatcher
import Levenshtein
import cv2
import time
import numpy as np

# Directories
expected_output_directory = 'cleaned/000'
actual_output_directory = 'outputs/py_cleaned'
trainSize = 25000
validateSize = 2500

def cleanImage(img):
     #NEED TO FINISH
    img =img 
    return img

def cleanTextFile(text):
    return text.replace('\n', ' ')
    
# METHOD 1:
# Similarity percentage using Levenshtein distance (edit distance), more robust for OCR
# Levenshtein distance outputs how similar, not dis-similar
def calculate_similarity_lev(text1, text2):
    text1 = cleanTextFile(text1)
    text2 = cleanTextFile(text2)
    print(f"Text1: {text1}")
    print(f"Text2: {text2}")
    
    distance = Levenshtein.distance(text1, text2)
    max_length = max(len(text1), len(text2))
    if max_length == 0:  # Avoids dividing by zero
        return 100.0
    return (1 - distance / max_length) * 100

def openCalculate(path):
    result = ''
    if os.path.exists(path):
                with open(path, 'r') as f:
                    result = f.read()
    return result

def prepTrainValid(type, size):
    for i in range(size):
        path =''
        if type == "train":
            path = "1/train_v2/train/"+train.loc[i, "FILENAME"]
        elif type =="validation":
            path = "1/validation_v2/validation/"+valid.loc[i, "FILENAME"]
        else:
             print("Wrong type")
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cleanImage(img)
        img = img/255 #Normalization
        return np.array(img).reshape(-1,256,64,1)


In [25]:
def loadDataAll(groundTruthDir, pyTesseractOutputTyped, pyTesseractOutputWritten, 
                kerasOCROutputTyped, kerasOCROutputWritten, easyOCROutputTyped, easyOCROutputWritten):
    count = 0
    pyTesseractTypedScore = 0
    pyTesseractWrittenScore = 0
    kerasOCRTypedScore = 0
    kerasOCRWrittenScore = 0
    easyOCRTypedScore = 0
    easyOCRWrittenScore = 0

    for filename in os.listdir(groundTruthDir):
        if filename.endswith('.txt'):
            # Construct file paths
            groundTruthPath = os.path.join(groundTruthDir, filename)
            pyTesseractTypedPath = os.path.join(pyTesseractOutputTyped, filename)
            pyTesseractWrittenPath = os.path.join(pyTesseractOutputWritten, filename)
            kerasOCRTypedPath = os.path.join(kerasOCROutputTyped, filename)
            kerasOCRWrittenPath = os.path.join(kerasOCROutputWritten, filename)
            easyOCRTypedPath = os.path.join(easyOCROutputTyped, filename)
            easyOCRWrittenPath = os.path.join(easyOCROutputWritten, filename)
    
            # Read file contents
            groundTruthText = openCalculate(groundTruthPath)
            pyTesseractTypedText = openCalculate(pyTesseractTypedPath)
            pyTesseractWrittenText = openCalculate(pyTesseractWrittenPath)
            kerasOCRTypedText = openCalculate(kerasOCRTypedPath)
            kerasOCRWrittenText = openCalculate(kerasOCRWrittenPath)
            easyOCRTypedText = openCalculate(easyOCRTypedPath)
            easyOCRWrittenText = openCalculate(easyOCRWrittenPath)

            # Calculate similarities
            pyTesseractTypedScoreLev = calculate_similarity_lev(pyTesseractTypedText, groundTruthText)
            pyTesseractWrittenScoreLev = calculate_similarity_lev(pyTesseractWrittenText, groundTruthText)
            kerasOCRTypedScoreLev = calculate_similarity_lev(kerasOCRTypedText, groundTruthText)
            kerasOCRWrittenScoreLev = calculate_similarity_lev(kerasOCRWrittenText, groundTruthText)
            easyOCRTypedScoreLev = calculate_similarity_lev(easyOCRTypedText, groundTruthText)
            easyOCRWrittenScoreLev = calculate_similarity_lev(easyOCRWrittenText, groundTruthText)
            
            print(f"File: {filename}")
            print(f"Similarity Percentage from Levenshtein with PyTesseract Typed: {pyTesseractTypedScoreLev:.2f}%")
            print(f"Similarity Percentage from Levenshtein with PyTesseract Written: {pyTesseractWrittenScoreLev:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Keras OCR Typed: {kerasOCRTypedScoreLev:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Keras OCR Written: {kerasOCRWrittenScoreLev:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Easy OCR Typed: {easyOCRTypedScoreLev:.2f}%")
            print(f"Similarity Percentage from Levenshtein with Easy OCR Written: {easyOCRWrittenScoreLev:.2f}%")
            
            # Accumulate scores
            count += 1
            pyTesseractTypedScore += pyTesseractTypedScoreLev
            pyTesseractWrittenScore += pyTesseractWrittenScoreLev
            kerasOCRTypedScore += kerasOCRTypedScoreLev
            kerasOCRWrittenScore += kerasOCRWrittenScoreLev
            easyOCRTypedScore += easyOCRTypedScoreLev
            easyOCRWrittenScore += easyOCRWrittenScoreLev            
        else:
            print(f"Expected output not found for {filename}")

    # Calculate averages and return results
    if count > 0:
        result = {
            "pytesseract Typed": pyTesseractTypedScore / count,
            "pytesseract Written": pyTesseractWrittenScore / count,
            "kerasOCR Typed": kerasOCRTypedScore / count,
            "kerasOCR Written": kerasOCRWrittenScore / count,
            "easyOCR Typed": easyOCRTypedScore / count,
            "easyOCR Written": easyOCRWrittenScore / count
        }
        return result
    else:
        return None

# Pytesseract

In [26]:
import pytesseract
from PIL import Image
import os

#Tesseract executable
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"

imageWritten = 'cleaned/000-written'
imageTyped = 'cleaned/000-typed'
imageNameWritten = '1/test_v2/test'

def processImagesPyTesseract(imageDirectory, outputDirectory, process):
    pyTesseractTimes = {}
    counter = 0
    for filename in os.listdir(imageDirectory):
        print(filename)
        if counter == 200:
            break
        if filename.endswith('.png') or filename.endswith('jpg'):
            image_path = os.path.join(imageDirectory, filename)

            # Open using PIL (Need to open with RGB and not BGR)
            img = Image.open(image_path)
            if process:
                img = cleanImage(img, "tesseract") 

            # Output file format
            output_file =outputDirectory + os.path.splitext(filename)[0] + '.txt'

            # Run Tesseract and writes into output file
            with open(output_file, 'w') as f:
                startTime = time.time()
                text = pytesseract.image_to_string(img, config="--oem 3 --psm 6") #better for written notes (oem 3 is best OCR engine [LSTM + Legacy], psm 6 assumes block text) 
                endTime = time.time()
                pyTesseractTimes[filename] = endTime - startTime
                f.write(text)

            print(f"Text extracted from {filename} and saved to {output_file}")
        counter += 1
    return pyTesseractTimes
#Uncomment to run PyTesseract on images
#print("Processing images in directory (PyTesseract):", imageWritten)
#pyTesseractWrittenTime= processImagesPyTesseract(imageWritten, "pyWritten", False)
#print("\nProcessing images in directory  (PyTesseract):", imageTyped)
#pyTesseractTypedTime=processImagesPyTesseract(imageTyped, "pyTyped", False)
temp = processImagesPyTesseract(imageNameWritten, "output-name/pytesseract/", False)


TEST_0001.jpg
Text extracted from TEST_0001.jpg and saved to output-name/pytesseract/TEST_0001.txt
TEST_0002.jpg
Text extracted from TEST_0002.jpg and saved to output-name/pytesseract/TEST_0002.txt
TEST_0003.jpg
Text extracted from TEST_0003.jpg and saved to output-name/pytesseract/TEST_0003.txt
TEST_0004.jpg
Text extracted from TEST_0004.jpg and saved to output-name/pytesseract/TEST_0004.txt
TEST_0005.jpg
Text extracted from TEST_0005.jpg and saved to output-name/pytesseract/TEST_0005.txt
TEST_0006.jpg
Text extracted from TEST_0006.jpg and saved to output-name/pytesseract/TEST_0006.txt
TEST_0007.jpg
Text extracted from TEST_0007.jpg and saved to output-name/pytesseract/TEST_0007.txt
TEST_0008.jpg
Text extracted from TEST_0008.jpg and saved to output-name/pytesseract/TEST_0008.txt
TEST_0009.jpg
Text extracted from TEST_0009.jpg and saved to output-name/pytesseract/TEST_0009.txt
TEST_0010.jpg
Text extracted from TEST_0010.jpg and saved to output-name/pytesseract/TEST_0010.txt
TEST_0011.

Keras OCR

In [ ]:
import os
import keras_ocr
import numpy as np
from PIL import Image
from typing import List

def processImagesKerasOcr(imageDirectory, outputDirectory):
    pipeline = keras_ocr.pipeline.Pipeline()
    
    # Get all image files from the folder
    imagePaths = []
    validExtensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')
    
    counter = 0
    for filename in os.listdir(imageDirectory):
        if counter == 200:
            break
        if filename.lower().endswith(validExtensions):
            print("Testing image: ", filename)
            imagePaths.append(os.path.join(imageDirectory, filename))
        counter += 1
    if not imagePaths:
        return
    
    images = [keras_ocr.tools.read(img_path) for img_path in imagePaths]
    prediction_groups = pipeline.recognize(images)
    
    results = []
    for predictions in prediction_groups:
        text = ' '.join([prediction[0] for prediction in predictions])
        results.append(text)

    for filename, predictions in zip(imagePaths, prediction_groups):
        text = ' '.join([prediction[0] for prediction in predictions])
        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_filepath = os.path.join(outputDirectory, txt_filename.split("\\")[-1])
        with open(txt_filepath, "w", encoding="utf-8") as txt_file:
            print("Writting into: ", txt_filepath)
            txt_file.write(text)
            txt_file.write(text)
    return #NEED TO ADD RESULT

# Example usage:
processImagesKerasOcr(imageNameWritten,'output-name/keras')


Looking for C:\Users\adam4\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\adam4\.keras-ocr\crnn_kurapan.h5
Testing image:  TEST_0001.jpg
Testing image:  TEST_0002.jpg
Testing image:  TEST_0003.jpg
Testing image:  TEST_0004.jpg
Testing image:  TEST_0005.jpg
Testing image:  TEST_0006.jpg
Testing image:  TEST_0007.jpg
Testing image:  TEST_0008.jpg
Testing image:  TEST_0009.jpg
Testing image:  TEST_0010.jpg
Testing image:  TEST_0011.jpg
Testing image:  TEST_0012.jpg
Testing image:  TEST_0013.jpg
Testing image:  TEST_0014.jpg
Testing image:  TEST_0015.jpg
Testing image:  TEST_0016.jpg
Testing image:  TEST_0017.jpg
Testing image:  TEST_0018.jpg
Testing image:  TEST_0019.jpg
Testing image:  TEST_0020.jpg
Testing image:  TEST_0021.jpg
Testing image:  TEST_0022.jpg
Testing image:  TEST_0023.jpg
Testing image:  TEST_0024.jpg
Testing image:  TEST_0025.jpg
Testing image:  TEST_0026.jpg
Testing image:  TEST_0027.jpg
Testing image:  TEST_0028.jpg
Testing image:  TEST_0029.jpg
Testing image:  TEST_